# 单无人机路径规划————最短路径的引导

（注意：运行本程序需要获得task3中生成的env_discretizing.pkl文件）

让我们将<a href="task3_SUAV_RL_Design.ipynb">无人机强化学习任务</a>中单无人机路径规划部分生成的环境文件加载进来

In [1]:
import sys
sys.path.append('..')
import UAVENVmudule_design as UE

import pickle
with open("env_discretizing.pkl", "rb") as file:
    env_discretizing = pickle.load(file)

对于此环境，本节我们用最短路径对无人机进行引导飞行。